In [21]:
# !pip3 install nltk

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import PorterStemmer
import re
# import nltk
# nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Hp
[nltk_data]     15\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

# Data Loading (Scrapping/Extraction)

In [16]:
def extract_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors (e.g., 404)
        soup = BeautifulSoup(response.content, 'html.parser')
        base_url = response.url  # Get the base URL after redirection
        links = [urljoin(base_url, link.get('href')) for link in soup.find_all('a', href=True)]
        return links
    except requests.RequestException as e:
        print(f"Error occurred while accessing {url}: {e}")
        return []

def extract_title_and_description(link):
    try:
        response = requests.get(link)
        response.raise_for_status()  # Raise an exception for HTTP errors (e.g., 404)
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.title.get_text() if soup.title else ""
        description_tag = soup.find('meta', {'name': 'description'})
        description = description_tag['content'] if description_tag else ""
        return title, description
    except requests.RequestException as e:
        print(f"Error occurred while accessing {link}: {e}")
        return "", ""
    except KeyError:
        return "", ""


def main():
    # Extract links from dawn.com and bbc.com
    dawn_links = extract_links('https://www.dawn.com/')
    bbc_links = extract_links('https://www.bbc.com/')
    
    # Print Dawn.com links
    print("Links from Dawn.com:")
    for link in dawn_links:
        print(link)
    
    # Print BBC.com links
    print("\nLinks from BBC.com:")
    for link in bbc_links:
        print(link)

    # Extract title and description for each link
    data = []
    for link in dawn_links + bbc_links:
        title, description = extract_title_and_description(link)
        data.append({'link': link, 'title': title, 'description': description})
    
    # Create Pandas DataFrame
    df = pd.DataFrame(data)
    
    # Save DataFrame to CSV
    df.to_csv('dataset_news.csv', index=False)
    # print("\nData saved to links_with_title_and_description.csv")


if __name__ == "__main__":
    main()


Links from Dawn.com:
https://epaper.dawn.com
https://www.dawnnews.tv/watch-live
https://www.dawnnews.tv
https://images.dawn.com
https://herald.dawn.com
https://aurora.dawn.com
https://cityfm89.com
https://www.dawn.com/advertise
https://educationexpo.dawn.com
https://www.dawn.com/events/supplements
http://classifieds.dawn.com/
https://obituary.dawn.com
https://www.dawn.com/
https://epaper.dawn.com
https://www.dawn.com
https://www.dawn.com/latest-news
https://www.dawn.com/trends/gaza-invasion
https://www.dawn.com/pakistan
https://www.dawn.com/opinion
https://www.dawn.com/business
https://www.dawn.com/world
https://www.dawn.com/entertainment
https://www.dawn.com/prism
https://www.dawn.com/sport
https://www.dawn.com/magazines
https://www.dawn.com/tech
https://www.dawn.com/videos
https://www.dawn.com/popular
https://www.dawn.com/newspaper
https://www.dawnrelief.com/
https://www.dawn.com
https://www.dawn.com/latest-news
https://www.dawn.com/trends/gaza-invasion
https://www.dawn.com/pakistan


# Data Transformation (Preprocessing/Cleaning)

In [32]:
# Load the dataset
news_links_dataset_v1 = pd.read_csv('dataset_news.csv')

# Remove duplicates
news_links_dataset_v2 = news_links_dataset_v1.drop_duplicates()

# Handle missing values (if any)
news_links_dataset_v3 = news_links_dataset_v2.dropna()

# Define a function to clean text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

# Apply the cleaning function to the 'description' column
news_links_dataset_v3['cleaned_description'] = news_links_dataset_v3['description'].apply(clean_text)

news_links_dataset_v3
news_links_dataset_v3.to_csv('dataset_news_new.csv', index=False)


C:\Users\Hp 15\AppData\Local\Temp\ipykernel_18444\3985252793.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_links_dataset_v3['cleaned_description'] = news_links_dataset_v3['description'].apply(clean_text)
